## Autonomously send real sensor values to TTN

**Prerequisites:** You must have followed the [getting started with TTN notebook](../ttn-getting-started/ttn-getting-started.ipynb) before starting this one.

> We also consider that the application id is **iotlab-lorawan** and the device id is **iotlab-node** but you'll have to use the ids corresponding to your application and device configured in TTN.

In this notebook, you will write an autonomous LoRaWAN application with the following specifications:
- The application automatically connects to TTN at startup using the OTAA activation and a datarate index of 5
- In case of an activation issue, an error message is printed to the console and the application returns
- A message is printed to the console to confirm the success of the activation
- A message is sent periodically (check the duty cycle!) to the TTN backend. This message will first contain a basic string and in a second phase, it will contain the values measured on the temperature and humidity sensors.
- A message is printed to the console each time the message is sent with success.

We provide a starting RIOT application with a [Makefile](Makefile) and a [main.c](./main.c). The [Makefile](Makefile) is already configured but you will have to edit the [main.c](./main.c) by following the instructions of this Notebook.

### Configure the TTN payload format

Before we start coding the application, we need to change how the data is displayed in the **Live Data** tab of your TTN application.

You can change this format of the payload used to display the content of an uplink message in the `Payload formatters>Uplink` tab:

<figure style="text-align:center">
    <img src="images/application-tab-payload-format.png" width="448" align="center"/><br/><br/>
    <figcaption><em>Application `Payload formatters` tab in the TTN console</em></figcaption>
</figure>

1. Select `Javascript` in the drop down button.
2. Copy the following code snippet in the `decoder` text edit:

```js
function decodeUplink(input) {
var message = "";

for  (var i=0; i < input.bytes.length; i++) {
    message += String.fromCharCode(input.bytes[i]);
}

return {
    data: {
      message: message,
    },
    warnings: [],
    errors: []
  };
}
```
3. Click on the `Save changes` button at the end of the page

### Implement an autonomous RIOT application

The LoRaWAN API of RIOT is documented online at http://doc.riot-os.org/group__pkg__semtech-loramac.html.

1. Edit the [Makefile](Makefile) file and add the `auto_init_loramac` module to
initiate the LoRa radio and the LoRaWAN stack.

```mk
USEMODULE += auto_init_loramac
```

Let's then edit the [main.c](./main.c).

2. Add the necessary includes under the comment `/* Add necessary include here */`:


```c
#include "net/loramac.h"     /* core loramac definitions */
#include "semtech_loramac.h" /* package API */
```

4. Declare the global descriptor for the loramac stack:


```c
semtech_loramac_t loramac;  /* The loramac stack descriptor */
```


4. Configure the identifiers (application and device) and the application key. You can find them in the device overview on TTN.


```c
static const uint8_t deveui[LORAMAC_DEVEUI_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
static const uint8_t appeui[LORAMAC_APPEUI_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
static const uint8_t appkey[LORAMAC_APPKEY_LEN] = { 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00 };
```

**note:** in the device overview on TTN, it's possible to switch the representation of the EUIs and key from an hexadecimal representation (the default) to a C byte array representation (the one that interests us here): use the `<>` button to switch from one to the other and keep the MSB order.

5. Configure the keys:

```c
    semtech_loramac_set_deveui(&loramac, deveui);
    semtech_loramac_set_appeui(&loramac, appeui);
    semtech_loramac_set_appkey(&loramac, appkey);
```

6. All devices are very close to the gateway in IoT-LAB, so we can configure a fast datarate. Let's use DR5:

```c
    semtech_loramac_set_dr(&loramac, 5);
```

7. Add the logic to join the network using the OTAA activation:

```c
    if (semtech_loramac_join(&loramac, LORAMAC_JOIN_OTAA) != SEMTECH_LORAMAC_JOIN_SUCCEEDED) {
        puts("Join procedure failed");
        return 1;
    }
    puts("Join procedure succeeded");
```

8. Finally, in the `while` loop (under the comment `/* send the message here */`), send the message or continue if the message couldn't be sent:

```c
        if (semtech_loramac_send(&loramac,
                                 (uint8_t *)message, strlen(message)) != SEMTECH_LORAMAC_TX_DONE) {
            printf("Cannot send message '%s'\n", message);
        }
        else {
            printf("Message '%s' sent\n", message);
        }
```

9. Save the changes in main.c using the `Ctrl + s` keyboard shortcut and then verify that the application builds correctly:

**It is normal if nothing is printed: there's normally no firmware running on the board. The firmware will be flashed during the next step**

**Keep this command running until the end of this notebook.**

4. Build and flash the application on the LoRa board:

In [ ]:
!make flash

Once the flashing is complete, in the terminal, you should see the messages corresponding to the join procedure ("Join procedure succeeded") and the messages sent every 20s.

If the device cannot join, verify the configured EUIs and the application key.

If everything works as expected, you should see the decoded "This is RIOT!" messages appear in the **Live Data** tab of your application:

<figure style="text-align:center">
    <img src="images/ttn-decoded-messages-received.png" width="1262" align="center"/><br/><br/>
    <figcaption><em>Decoded messages received in the TTN console</em></figcaption>
</figure>

### Read the sensor values

Let's add the final step: read some sensor values from LM75a temperature sensor.

Let's now edit the [Makefile](Makefile) and [main.c](main.c) to read this sensor.

1. To be able to use the _lm75a_ sensor with RIOT, you must first load the corresponding RIOT module in the application's [Makefile](Makefile). Add the following line to the Makefile (under the addition of the `xtimer` module):


```mk
USEMODULE += lm75a
```

All dependency modules required by the sensor driver will be loaded automatically during compilation, in particular the I2C bus driver used by the board to communicate with the sensor.

2. In the [main.c](main.c) file, Add the necessary header inclusions first (just below `#includes "semtech_loramac.h"`):


```c
#include "lm75.h"
#include "lm75_params.h"
```

3. Declare the variable containing the driver descriptor of the sensor (below the declaration of the `loramac` variable):

```c
static lm75_t lm75;
```

4. At the very beginning of the `main` function, add this sensor driver initialization sequence:

```c
    if (lm75_init(&lm75, &lm75_params[0]) != LM75_SUCCESS) {
        puts("Sensor initialization failed");
        return 1;
    }
```

5. Finally, add the following code at the beginning of the `while` loop to read the sensor data. You can also delete the line `char *message = "This is RIOT!"`
which is now useless:

```c
        /* do some measurements */
        int temperature = 0;
        if (lm75_get_temperature(dev, &temperature)!= LM75_SUCCESS) {
            puts("Cannot read temperature!");
        }

        char message[32];
        sprintf(message, "T:%d.%dC",
                (temperature / 1000), (temperature % 1000));
        printf("Sending message '%s'\n", message);
```

6. Build and flash the new application to the LoRa board of your experiment:

In [ ]:
!make flash

Now you can switch back to the terminal that is connected to the serial port of the LoRa board: **a new join procedure should be performed and the sensor data are sent every 20s to TTN**.

<figure style="text-align:center">
    <img src="images/ttn-sensor-data.png" width="1332" align="center"/><br/><br/>
    <figcaption><em>Sensor data received in the TTN console</em></figcaption>
</figure>
